In [1]:
import torch
import torch.nn as nn
import torch.nn.utils as nn_utils
import torch.nn.functional as F
import torch.optim as optim

In [4]:
class NnA2CController(nn.Module):
    def __init__(self, frames_shape):
        """Setus up all of the neural networks necessary for the controller to work
        
        Parameters
        ----------
        frames_shape
            the shape of the multidimensional carla data fetched from an agent's sensor
        
        Attributes
        ----------
        conv_net: nn.Sequential
            a neural network responsible for extracting the current state of the environment, whose output is meant to serve as input for policy and critic networks
        actor_net: nn.Sequential
            a neural network responsible for the current agent's policy
        critic_net: nn.Sequential
            a neural network responsible for approximating the advantage function regarding the action space
        
        Methods
        -------
        TBD
        """
        super(NnA2CController, self).__init__()
        
        #lenet inspired net upscaled due to carla frames being bigger than minst digits ;)
        self.conv_net = nn.Sequential(
            nn.Conv2d(in_channels=frames_shape, out_channels=128, kernel_size=8, stride=4),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=128, out_channels=64, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=4),
            nn.Conv2d(in_channels=64, out_channels=256, kernel_size = 8, stride = 4),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=256, out_channels=120, kernel_size=5, stride=1),
            nn.Tanh()
        )
        
        conv_out_size = int(np.prod(self.conv_net(torch.zeros(1, *shape)).size()))
        
        self.actor_net = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, 2) #2 returned values being the action taken, which consists of the gas/break pedal and steering angle suggested for steering
        )

        self.critic_net = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )
    
        
    def forward(self, sensor_data):
        conv_output = self.conv_net(sensor_data)
        return self.actor_net(conv_output), self.critic_net(conv_output)
    
    def control(self, state):

        #leaving the following state elements here, but intend to use just camera data for starters
        location = state['location']
        x, y = location[0], location[1]
        v = state['velocity'] # km / h #how to get speed?????????
        ψ = np.radians(state['yaw']) #adding 180 as carla returns yaw degrees in (-180, 180) range

        steer, throttle, = self.forward(state[''])

        actions = {
            'steer': steer,
            'gas_brake': throttle,
        }

        return actions